In [1]:
#setting up API backend and useful packages
import requests
import pandas as pd
import random
import time
from ast import literal_eval
import glob

In [2]:
pip install git+https://github.com/Dewey-Data/deweydatapy.git #installs Dewey Data package for convenience

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#installs'


In [3]:
import deweydatapy as ddp

## API Setup

In [5]:
#individual key (get your own!)
API_KEY = '7LQsijkY.KX88klG0wxYG3kNa7P4sgWDHLJaf2czgQTtR3PDb39mfCoX9S9PHit3B'
#dec 24 to feb 25
url25 = f"https://app.deweydata.io/external-api/v3/products/ff6af5ab-fe02-4ea3-9a24-5fad03700b38/files"
#jan 24
url24 = f"https://app.deweydata.io/external-api/v3/products/712cd497-9ff8-45a6-b15b-b14ea3ba5146/files"

In [6]:
ddp.download_files0(API_KEY, url24, "C:/Users/rf2694",
                    start_date = '2024-01-01', end_date = '2024-01-31', skip_exists=True) #be sure to change the directory!

Files information collection completed.
 
Files information summary ---------------------------------------
Total number of pages: 1
Total number of files: 3
Total files size (MB): 545.79
Average single file size (MB): 181.93
Date partition column: DATE_RANGE_START
Expires at: 2025-02-27T19:29:51.349Z
-----------------------------------------------------------------
Start downloading...
 
File already exists: C:/Users/rf2694/Neighborhood_Patterns_NY_Jan_24-0-DATE_RANGE_START-2024-01-01.csv
Skipping...
File already exists: C:/Users/rf2694/Neighborhood_Patterns_NY_Jan_24-1-DATE_RANGE_START-2024-01-01.csv
Skipping...
File already exists: C:/Users/rf2694/Neighborhood_Patterns_NY_Jan_24-2-DATE_RANGE_START-2024-01-01.csv
Skipping...
 
Download completed.


In [7]:
ddp.download_files0(API_KEY, url25, "C:/Users/rf2694",
                    start_date = '2025-01-01', end_date = '2025-01-31', skip_exists=True) #be sure to change the directory!

Files information collection completed.
 
Files information summary ---------------------------------------
Total number of pages: 1
Total number of files: 3
Total files size (MB): 437.46
Average single file size (MB): 145.82
Date partition column: DATE_RANGE_START
Expires at: 2025-02-27T19:29:52.098Z
-----------------------------------------------------------------
Start downloading...
 
File already exists: C:/Users/rf2694/Neighborhood_Patterns_NY_Dec_24_to_Feb_25-0-DATE_RANGE_START-2025-01-01.csv
Skipping...
File already exists: C:/Users/rf2694/Neighborhood_Patterns_NY_Dec_24_to_Feb_25-1-DATE_RANGE_START-2025-01-01.csv
Skipping...
File already exists: C:/Users/rf2694/Neighborhood_Patterns_NY_Dec_24_to_Feb_25-2-DATE_RANGE_START-2025-01-01.csv
Skipping...
 
Download completed.


In [8]:
#brief tutorial on glob: https://www.geeksforgeeks.org/how-to-use-glob-function-to-find-files-recursively-in-python/
#code reference: https://python.plainenglish.io/glob-and-pandas-working-in-unison-304242627e78
frame = []
for name in glob.glob('Neighborhood_Patterns_NY_Dec_24_to_Feb_25*'):
    frame.append(pd.read_csv(name, usecols=['AREA','STOPS_BY_EACH_HOUR','POPULARITY_BY_EACH_HOUR','STOPS_BY_DAY','DATE_RANGE_START','DATE_RANGE_END']))

data25 = pd.concat(frame)

In [9]:
frame = []
for name in glob.glob('Neighborhood_Patterns_NY_Jan_24*'):
    frame.append(pd.read_csv(name, usecols=['AREA','STOPS_BY_EACH_HOUR','POPULARITY_BY_EACH_HOUR','STOPS_BY_DAY','DATE_RANGE_START','DATE_RANGE_END']))

data24 = pd.concat(frame)

## Defining the Congestion Pricing Zone

In [11]:
#geopandas & read zip file with QGIS output
import geopandas as gp #install if needed
crzone = gp.read_file("zip:///Users/rf2694/Documents/QGIS/crz2019.zip")[['GEOID','geometry']] #modify file path to locate congestion zone shapefile
crzone = crzone.astype({'GEOID': 'int64'}) #for merging purposes
crzone.head()

,GEOID,geometry
0,360610078001,"POLYGON ((-73.97584 40.74886, -73.97497 40.748..."
1,360610050004,"POLYGON ((-73.98878 40.73397, -73.98829 40.734..."
2,360610044007,"POLYGON ((-73.98202 40.73201, -73.98156 40.732..."
3,360610063004,"POLYGON ((-73.99919 40.73413, -73.99902 40.734..."
4,360610064005,"POLYGON ((-73.9809 40.73789, -73.9785 40.73679..."


## Limiting Data to the Congestion Pricing Zone

In [13]:
#use pandas merge to filter Dewey data by CZ GEOID (function also works for geopandas)
#reference for pandas merge: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html
bigData25 = crzone.merge(right=data25, left_on='GEOID', right_on='AREA')
bigData24 = crzone.merge(right=data24, how='left', left_on='GEOID', right_on='AREA')

## Stops by Day

In [76]:
#STOPS BY DAY DATASET (2025)
dataByDay25 = bigData25[["GEOID","geometry","DATE_RANGE_START","DATE_RANGE_END","STOPS_BY_DAY"]]
#dataByDay25 = dataByDay25.dropna()
dataByDay25['STOPS_BY_DAY'] = dataByDay25['STOPS_BY_DAY'].apply(literal_eval) #changes type from string to actual list lol
dataByDate25 = dataByDay25.explode('STOPS_BY_DAY') #boom!

day25 = pd.date_range(start=bigData25['DATE_RANGE_START'][0], end=bigData25['DATE_RANGE_END'][0]).strftime("%Y-%m-%d").tolist()
day25 = day25[0:31] #every date in january 2025, not including feb 1
dates25 = day25 * len(dataByDay25)
dataByDate25['DATE']=dates25
dataByDateClean25 = dataByDate25[["GEOID","geometry","DATE", "STOPS_BY_DAY"]]
dataByDateClean25 = dataByDateClean25.astype({'STOPS_BY_DAY': 'int64'})
print(dataByDateClean25[0:11])

          GEOID                                           geometry  \
0  360610078001  POLYGON ((-73.97584 40.74886, -73.97497 40.748...   
0  360610078001  POLYGON ((-73.97584 40.74886, -73.97497 40.748...   
0  360610078001  POLYGON ((-73.97584 40.74886, -73.97497 40.748...   
0  360610078001  POLYGON ((-73.97584 40.74886, -73.97497 40.748...   
0  360610078001  POLYGON ((-73.97584 40.74886, -73.97497 40.748...   
0  360610078001  POLYGON ((-73.97584 40.74886, -73.97497 40.748...   
0  360610078001  POLYGON ((-73.97584 40.74886, -73.97497 40.748...   
0  360610078001  POLYGON ((-73.97584 40.74886, -73.97497 40.748...   
0  360610078001  POLYGON ((-73.97584 40.74886, -73.97497 40.748...   
0  360610078001  POLYGON ((-73.97584 40.74886, -73.97497 40.748...   
0  360610078001  POLYGON ((-73.97584 40.74886, -73.97497 40.748...   

         DATE  STOPS_BY_DAY  
0  2025-01-01           142  
0  2025-01-02           222  
0  2025-01-03           160  
0  2025-01-04            44  
0  2025-0

C:\Users\rf2694\AppData\Local\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [16]:
#write to shapefile
#note that variable names >10 characters will get cut off. just rename in QGIS idk
dataByDateClean25.to_file("J25_daily.shp")

C:\Users\rf2694\AppData\Local\Temp\ipykernel_10320\926012306.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  dataByDateClean25.to_file("J25_daily.shp")
C:\Users\rf2694\AppData\Local\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'STOPS_BY_DAY' to 'STOPS_BY_D'
  ogr_write(


In [78]:
#STOPS BY DAY DATASET (2024)
dataByDay24 = bigData24[["GEOID","geometry","DATE_RANGE_START","DATE_RANGE_END","STOPS_BY_DAY"]]
dataByDay24 = dataByDay24.dropna()
dataByDay24['STOPS_BY_DAY'] = dataByDay24['STOPS_BY_DAY'].apply(literal_eval) #changes type from string to actual list lol
dataByDate24 = dataByDay24.explode('STOPS_BY_DAY') #boom!

day24 = pd.date_range(start=bigData24['DATE_RANGE_START'][0], end=bigData24['DATE_RANGE_END'][0]).strftime("%Y-%m-%d").tolist()
day24 = day24[0:31] #every date in january 2024, not including feb 1
dates24 = day24 * len(dataByDay24)
dataByDate24['DATE']=dates24
dataByDateClean24 = dataByDate24[["GEOID","geometry","DATE", "STOPS_BY_DAY"]]
dataByDateClean24 = dataByDateClean24.astype({'STOPS_BY_DAY': 'int64'})
print(dataByDateClean24[0:11])

          GEOID                                           geometry  \
0  360610078001  POLYGON ((-73.97584 40.74886, -73.97497 40.748...   
0  360610078001  POLYGON ((-73.97584 40.74886, -73.97497 40.748...   
0  360610078001  POLYGON ((-73.97584 40.74886, -73.97497 40.748...   
0  360610078001  POLYGON ((-73.97584 40.74886, -73.97497 40.748...   
0  360610078001  POLYGON ((-73.97584 40.74886, -73.97497 40.748...   
0  360610078001  POLYGON ((-73.97584 40.74886, -73.97497 40.748...   
0  360610078001  POLYGON ((-73.97584 40.74886, -73.97497 40.748...   
0  360610078001  POLYGON ((-73.97584 40.74886, -73.97497 40.748...   
0  360610078001  POLYGON ((-73.97584 40.74886, -73.97497 40.748...   
0  360610078001  POLYGON ((-73.97584 40.74886, -73.97497 40.748...   
0  360610078001  POLYGON ((-73.97584 40.74886, -73.97497 40.748...   

         DATE  STOPS_BY_DAY  
0  2024-01-01            83  
0  2024-01-02           205  
0  2024-01-03           220  
0  2024-01-04           246  
0  2024-0

In [18]:
#write to shapefile
#note that variable names >10 characters will get cut off. just rename in QGIS idk
dataByDateClean24.to_file("J24_daily.shp")

C:\Users\rf2694\AppData\Local\Temp\ipykernel_10320\2719369358.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  dataByDateClean24.to_file("J24_daily.shp")
C:\Users\rf2694\AppData\Local\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'STOPS_BY_DAY' to 'STOPS_BY_D'
  ogr_write(


## General Analysis

In [80]:
#reindex to make slicing feasible
#indices were likely messed up by exploding previously
new_index = range(0,len(dataByDateClean24))
dataByDateClean24.index = new_index

new_index = range(0,len(dataByDateClean25))
dataByDateClean25.index = new_index

In [82]:
dataByDateClean25['DATE'] = pd.to_datetime(dataByDateClean25['DATE'], format='%Y-%m-%d')
dataByDateClean24['DATE'] = pd.to_datetime(dataByDateClean24['DATE'], format='%Y-%m-%d')

In [84]:
#add day of week
weekday = []
for i in range(0,len(dataByDateClean24)):
    weekday += [dataByDateClean24['DATE'][i].strftime('%A')]

dataByDateClean24['dow'] = weekday
dataByDateClean24.head()

,GEOID,geometry,DATE,STOPS_BY_DAY,dow
0,360610078001,"POLYGON ((-73.97584 40.74886, -73.97497 40.748...",2024-01-01,83,Monday
1,360610078001,"POLYGON ((-73.97584 40.74886, -73.97497 40.748...",2024-01-02,205,Tuesday
2,360610078001,"POLYGON ((-73.97584 40.74886, -73.97497 40.748...",2024-01-03,220,Wednesday
3,360610078001,"POLYGON ((-73.97584 40.74886, -73.97497 40.748...",2024-01-04,246,Thursday
4,360610078001,"POLYGON ((-73.97584 40.74886, -73.97497 40.748...",2024-01-05,198,Friday


In [86]:
#add day of week
weekday = []
for i in range(0,len(dataByDateClean25)):
    weekday += [dataByDateClean25['DATE'][i].strftime('%A')]

dataByDateClean25['dow'] = weekday

In [88]:
#creates "day" category to specify day separate from year for later merging
#reference: https://stackoverflow.com/questions/39908314/slice-all-strings-in-a-list
dataByDateClean25 = dataByDateClean25.astype({'DATE': 'string'})
dataByDateClean25['day']= [word[5:] for word in dataByDateClean25['DATE']]
dataByDateClean24 = dataByDateClean24.astype({'DATE': 'string'})
dataByDateClean24['day']= [word[5:] for word in dataByDateClean24['DATE']]

In [90]:
#remove zeroes (rows with no stops or data)
dataByDateClean24 = dataByDateClean24[dataByDateClean24['STOPS_BY_DAY'] != 0]
dataByDateClean25 = dataByDateClean25[dataByDateClean25['STOPS_BY_DAY'] != 0]

In [92]:
# average per day per zone and clean things up
#https://stackoverflow.com/questions/53549492/joining-two-pandas-dataframes-based-on-multiple-conditions
zoneavg = dataByDateClean25.merge(dataByDateClean24, how='inner', on=['GEOID','day'])
zoneavg = zoneavg[['GEOID','geometry_x','day','STOPS_BY_DAY_x','STOPS_BY_DAY_y','dow_x','dow_y']]
zoneavg = zoneavg.rename(columns={'STOPS_BY_DAY_x':'stops25','STOPS_BY_DAY_y':'stops24','geometry_x':'geom','dow_x':'dow25','dow_y':'dow24'})
zoneavg['percchange'] = zoneavg['stops25']/zoneavg['stops24']*100 -100
zoneavg.head()

,GEOID,geom,day,stops25,stops24,dow25,dow24,percchange
0,360610078001,"POLYGON ((-73.97584 40.74886, -73.97497 40.748...",01-01,142,83,Wednesday,Monday,71.084337
1,360610078001,"POLYGON ((-73.97584 40.74886, -73.97497 40.748...",01-02,222,205,Thursday,Tuesday,8.292683
2,360610078001,"POLYGON ((-73.97584 40.74886, -73.97497 40.748...",01-03,160,220,Friday,Wednesday,-27.272727
3,360610078001,"POLYGON ((-73.97584 40.74886, -73.97497 40.748...",01-04,44,246,Saturday,Thursday,-82.113821
4,360610078001,"POLYGON ((-73.97584 40.74886, -73.97497 40.748...",01-05,100,198,Sunday,Friday,-49.494949


In [94]:
#set weekend/weekday
day25 = []
day24 = []
for i in range(0,len(zoneavg)):
    if zoneavg['dow25'][i] in ['Saturday', 'Sunday']:
        day25 += ['weekend']
    else:
            day25 += ['weekday']
    if zoneavg['dow24'][i] in ['Saturday', 'Sunday']:
            day24 += ['weekend']
    else:
            day24 += ['weekday']

zoneavg['day25'] = day25
zoneavg['day24'] = day24

In [96]:
#for better comparison by weekend/weekday rather than specific date, make two datasets
zones24 = zoneavg[['GEOID','geom','stops24','dow24','day24']]
zones24.rename(columns={'day24':'day'},inplace=True)
zones25 = zoneavg[['GEOID','geom','stops25','dow25','day25']]
zones25.rename(columns={'day25':'day'},inplace=True)

C:\Users\rf2694\AppData\Local\Temp\ipykernel_10320\2468443686.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zones24.rename(columns={'day24':'day'},inplace=True)
C:\Users\rf2694\AppData\Local\Temp\ipykernel_10320\2468443686.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zones25.rename(columns={'day25':'day'},inplace=True)


In [98]:
#total stops in each zone
ztotal24 = zones24.groupby(by=['GEOID'])['stops24'].sum()
ztotal24.to_csv("zonetotal_24.csv")

ztotal25 = zones25.groupby(by=['GEOID'])['stops25'].sum()
ztotal25.to_csv("zonetotal_25.csv")

In [100]:
totalChangeZoned = ztotal25/ztotal24
totalChangeZoned.to_csv('totalchange.csv')

In [102]:
zoneavgClean = zoneavg[(zoneavg.stops24 != 0) & (zoneavg.stops25 != 0)] #removes values of -100% change and inf (both issues with missing values)
zoneavgClean.to_file("zonechangesClean.shp")
zoneavgClean.to_csv("zonechangesClean.csv")

## Depreciated

In [ ]:
#methods for filtering: https://www.kdnuggets.com/2022/12/five-ways-conditional-filtering-pandas.html
filteredbigDataUS = bigDataUS
filteredbigDataUS['DATE_RANGE_START'] = pd.to_datetime(bigDataUS['DATE_RANGE_START'], format='%Y-%m-%d %H:%M:%S.%f')
filteredbigDataUS['DATE_RANGE_END'] = pd.to_datetime(bigDataUS['DATE_RANGE_END'], format='%Y-%m-%d %H:%M:%S.%f')
filteredbigDataUS = filteredbigDataUS.query('GEOID.between(360610000000,360620000000) and DATE_RANGE_START >= "2025-01-01"')
filteredbigDataUS.head()

In [ ]:
#STOPS BY DAY DATASET (Filtered from US)
dataByDayUS = filteredbigDataUS[["GEOID","geometry","DATE_RANGE_START","DATE_RANGE_END","STOPS_BY_DAY"]]
dataByDayUS = dataByDayUS.dropna()
dataByDayUS['STOPS_BY_DAY'] = dataByDayUS['STOPS_BY_DAY'].apply(literal_eval) #changes type from string to actual list lol
dataByDateUS = dataByDayUS.explode('STOPS_BY_DAY') #boom!

dayUS = pd.date_range(start=bigDataUS['DATE_RANGE_START'][2], end=filteredbigDataUS['DATE_RANGE_END'][2]).strftime("%Y-%m-%d").tolist()
dayUS = dayUS[0:31] #every date in january 2025, not including feb 1
datesUS = dayUS * len(dataByDayUS)
dataByDateUS['DATE']=datesUS
dataByDateCleanUS = dataByDateUS[["GEOID","geometry","DATE", "STOPS_BY_DAY"]]
dataByDateCleanUS = dataByDateCleanUS.astype({'STOPS_BY_DAY': 'int64'})
print(dataByDateCleanUS[0:11])

In [ ]:
#STOPS BY DAY DATASET (merged with NY block group map)
filteredbigDataNY = bigDataNY
filteredbigDataNY['DATE_RANGE_START'] = pd.to_datetime(bigDataNY['DATE_RANGE_START'], format='%Y-%m-%d %H:%M:%S.%f')
filteredbigDataNY['DATE_RANGE_END'] = pd.to_datetime(bigDataNY['DATE_RANGE_END'], format='%Y-%m-%d %H:%M:%S.%f')
filteredbigDataNY = filteredbigDataNY.query('GEOID.between(360610000000,360620000000) and DATE_RANGE_START >= "2024-12-01"')

dataByDayNY = filteredbigDataNY[["GEOID","geometry","DATE_RANGE_START","DATE_RANGE_END","STOPS_BY_DAY"]]
dataByDayNY = dataByDayNY.dropna()
dataByDayNY['STOPS_BY_DAY'] = dataByDayNY['STOPS_BY_DAY'].apply(literal_eval) #changes type from string to actual list lol
dataByDateNY = dataByDayNY.explode('STOPS_BY_DAY') #boom!

dayNY = pd.date_range(start=filteredbigDataNY['DATE_RANGE_START'][3697], end=filteredbigDataNY['DATE_RANGE_END'][3697]).strftime("%Y-%m-%d").tolist()
dayNY = dayNY[0:31] #every date in january 2025, not including feb 1
datesNY = dayNY * len(dataByDayNY)
dataByDateNY['DATE']=datesNY
dataByDateCleanNY = dataByDateNY[["GEOID","geometry","DATE", "STOPS_BY_DAY"]]
dataByDateCleanNY = dataByDateCleanNY.astype({'STOPS_BY_DAY': 'int64'})
print(dataByDateCleanNY[0:11])

In [ ]:
#trying to get day by day weather data to reduce differences (thanks Dane)
#url = 'https://www.wunderground.com/history/monthly/us/ny/new-york-city/KLGA/date/2025-1'
#html = requests.get(url).content
df_list = pd.read_html("weather24.htm")
df = df_list[-1]
print(df)
df.to_csv('my data.csv')

### Spend Patterns

In [ ]:
urlSpend = f"https://app.deweydata.io/external-api/v3/products/eb6e748a-0fdd-4bc7-9dd7-bbed0890948d/files"

In [ ]:
ddp.download_files0(API_KEY, urlSpend, "C:/Users/rf2694",
                    start_date = '2024-01-01', end_date = '2024-01-31', skip_exists=True) #be sure to change the directory!
#ddp.download_files0(API_KEY, urlSpend, "C:/Users/rf2694", start_date = '2025-01-01', end_date = '2025-01-31', skip_exists=True) #be sure to change the directory!

In [ ]:
frame = []
for name in glob.glob('Spend_Patterns_Entire_US-?-SPEND_DATE_RANGE_START-2024*'):
    frame.append(pd.read_csv(name, usecols=['PLACEKEY','LOCATION_NAME','POPULARITY_BY_EACH_HOUR','SPEND_BY_DAY','SPEND_DATE_RANGE_START','SPEND_DATE_RANGE_END']))

spend24 = pd.concat(frame)